# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [2]:
# Only needed for Udacity workspace
import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
# TODO: Import the necessary libs
import os
import chromadb

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage, BaseMessage
from lib.tooling import tool
from lib.state_machine import Run
from lib.vector_db import VectorStoreManager
from lib.rag import RAG

from typing import List, Dict, Optional, Any, Union
from chromadb.utils import embedding_functions
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from tavily import TavilyClient
from datetime import datetime

In [4]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [5]:
# Helper functions and classes 
class EvaluationReport(BaseModel):
    useful: bool = Field(..., description="Whether retrieved docs are sufficient to answer the question.")
    description: str = Field(..., description="Detailed reasoning and actionable next steps.")
    missing_info: Optional[List[str]] = Field(
        default=None,
        description="Bullet list of missing facts needed to answer well (if any)."
    )
    recommended_action: Optional[str] = Field(
        default=None,
        description="One concrete action: eg 'use_web_search', 'rewrite query'."
    )

def print_messages(messages: List[BaseMessage]):
    for m in messages: 
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

def stringify_docs(retrieved_docs: List[Union[str, Dict[str, Any]]]) -> str:
    """
    Turn docs into a compact string for the judge.
    """
    chunks = []
    for i, d in enumerate(retrieved_docs, start=1):
        if isinstance(d, str):
            text = d
        else:
            # common fields you might have (adjust as needed)
            # e.g. {"name":..., "platform":..., "year":..., "description":...}
            text = json.dumps(d, ensure_ascii=False)
        text = text.strip()
        chunks.append(f"[Doc {i}] {text}")
    return "\n".join(chunks) if chunks else "(no documents provided)"

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [6]:
# Initialize the VectorStore needed for the Video Game RAG 
# load "chromadb" from file system and get vector store 
db = VectorStoreManager(OPENAI_API_KEY, persistent_client=True, dir_path="chromadb")
games_vector_store = db.get_or_create_store("udaplay_games_collection")

```
TODO: Create retrieve_game tool
It should use chroma client and collection you created
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")
Tool Docstring:
   Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry. 

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
```

In [7]:
@tool
def search_video_game_collection(query):
    """
    Semantic search: Search the vector database for relevant information on video games. Returns data for answering or evaluation. 
    
    Source: Video game JSON files
    ```
        The data represents video game information of the top 15 games of all times.  
    ```
    You'll receive results as list. Each element contains:
    - Name: Name of the Game
    - Description: details of the video game 
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - YearOfRelease: Year when that game was released for that platform

    args:
        query (str): Search query is a search string to query the vector DB. Transform the user question into a relevant search string query. 

    Example: 
        >>> search_video_game_collection("fighting game")
        >>> result = [[{'Genre': 'Fighting',
           'Description': 'A crossover fighting game featuring characters from various Nintendo franchises battling it out in dynamic arenas.',
           'Publisher': 'Nintendo',
           'Platform': 'GameCube',
           'YearOfRelease': 2001,
           'Name': 'Super Smash Bros. Melee'},
          {'Publisher': 'Sony Interactive Entertainment',
           'Genre': 'Action-adventure',
           'Name': "Marvel's Spider-Man",
           'Platform': 'PlayStation 4',
           'YearOfRelease': 2018,
           'Description': 'An open-world superhero game that lets players swing through New York City as Spider-Man, battling iconic villains.'},
          {'YearOfRelease': 2010,
           'Description': 'A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.',
           'Name': 'Gran Turismo 5',
           'Genre': 'Racing',
           'Platform': 'PlayStation 3',
           'Publisher': 'Sony Computer Entertainment'}]]
    """
    vector_db_query = games_vector_store.query(query)
    return vector_db_query["metadatas"]

#### Evaluate Retrieval Tool

```
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result
```

In [8]:
@tool 
def evalutate_retrieval_tool(user_question: str, retrieved_docs: List[Any]): 
    """
    Evaluate the usability of the retrived information to answering the user's question

    args:
        user_question (str): User's question 
        retrieved_docs (List): Retrieved information 

    Example: 
      >> evalutate_retrieval_tool(user_question="What is the best video game for driving?",
                                     retrieved_docs = [{"Platform":"PlayStation 3","Description":"A 
                                         comprehensive racing simulator featuring a vast selection of vehicles and tracks, 
                                         with realistic driving physics.","Genre":"Racing","YearOfRelease":2010,
                                         "Publisher":"Sony Computer Entertainment","Name":"Gran Turismo 5"}, ...] )
      >> returns {
          "useful": true,
          "description": "The retrieved documents provide a selection of notable driving video games, including...",
          "missing_info": [
            "More recent driving games released after 2017",
            "User preferences (e.g., realism vs. arcade style)",
            "Critical reception or reviews of the games"
          ],
          "recommended_action": "use_web_search"
        }
    """
    docs_text = str(retrieved_docs) 

    judge_prompt = (
        "Your task is to evaluate if the documents are enough to respond to a user query question."
        "Give a detailed explanation, so it's possible to take an action to accept it or not."
         f"Question: {user_question}"
         f"Retrieved Documents: {docs_text}"
    )

    # Query llm-as-a-judge
    llm_judge = LLM() 
    judge_response = llm_judge.invoke(
        input = judge_prompt, response_format = EvaluationReport
    )
    
    return judge_response

#### Game Web Search Tool

```
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry.
```

In [19]:
@tool
def web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web using Tavily API. Use this to find the most updated web search results. 
    args:
        query (str): Search query representing a question about the video game industry 
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)

    Example: 
        >> web_search("top games 2027")
        >> {'answer': "The top games for 2027 include Marvel's Blade, The Witcher 4, and Grand Theft Auto VI. These titles are highly anticipated and expected to be major releases.",
            'results': [{'url': 'https://www.youtube.com/watch?v=aX9X3C81SsQ',
                'title': 'TOP 10 NEW Upcoming Games of 2027',
                'content': "SUBSCRIBE ►►  ◄◄\n\n▬▬▬▬▬▬▬▬▬▬▬▬▬ ⬇ More info ⬇ ▬▬▬▬▬▬▬▬▬▬▬▬▬▬\n\n00:00 - Marvel's Blade\n01:17 - Welcome to Brightville\n04:13 - Project EVILBANE\n05:46 - Chronicles Medieval\n07:25 - Elusive\n09:00 - Grand Theft Auto VI\n11:10 - Frostpunk 1886\n11:29 - The Witcher 4\n14:07 - Intergalactic The Heretic Prophet\n16:22 - The Song of the Nightrider\n\n▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬\n\n● LET ME HELP REACH 500K: \n♥ LIKE IT? SHARE IT! [...] close and savor the moment. You raise my hopes and don't even know it. In a life that's hardside down, somehow you turn me around. So stay alive. [Music] We are [Music] Frostpunk 1886 It's coming. It's coming. Brace yourselves. Save yourselves. Save The Witcher 4 Nay. Nay, you can't. It is tradition. Chosen, she is by the gods. No, you chose her. [Music] Sh back to the village. Go now. Nay, I must save them. Save the village. Save yourself. Run to shelter to your father. Run. Kill us all. [...] ### Description\n2264 views\nPosted: 21 Oct 2025\nTOP 10 NEW Upcoming Games of 2027 highlights the most anticipated titles set to define the future of gaming. Featuring fresh gameplay details, developer insights, and the latest news for each game, this video covers everything from massive open-world adventures and next-gen RPGs to ambitious indie projects pushing creative boundaries. Get a quick, detailed look at the most exciting games coming in 2027 - all in one concise, informative roundup.",
                'score': 0.8684817,
                'raw_content': None}, ...]
            'search_metadata': {'timestamp': '2025-12-13T18:41:21.668819',
            'query': 'top games 2027'}, 
            'urls_for_citation': [{'title': 'TOP 10 NEW Upcoming Games of 2027 - YouTube',  'url': 'https://www.youtube.com/watch?v=aX9X3C81SsQ'}]
           }
    """
    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )

    # Extract URLs seperately 
    urls = []
    for result in search_result["results"]:
        urls.append({"title": result["title"], "url": result["url"]})
        
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }, 
        "urls_for_citation": urls
    }
    return formatted_results

### Agent

In [21]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed
agentic_rag = Agent(
    model_name="gpt-4o-mini",
    temperature = 0,
    tools=[search_video_game_collection, web_search, evalutate_retrieval_tool],
    instructions=(
        "You are an Agentic RAG assistant that specializes in content about the video game industry."
        "You can intelligently decide which tools to use to answer user questions."
        "You have access to 3 tools: search_video_game_collection, web_search and evalutate_retrieval_tool"
        "When answering, first retrieve internal docs via search_video_game_collection, then evaluate results with evaluate_retrieval_tool."
        "If not useful, take the recommended actions in order: (web_search, rewrite query or use foundational knowledge)."
        "Reason about about the response, change the query and call the tool again if needed in order to get better results."
        "Always explain your reasoning for tool selection and provide comprehensive answers."
        "When the tool web_search is used, response must have citations and end with a 'Sources:' section listing the URLs in a numbered format."
        )
)

In [22]:
run_2 = agentic_rag.invoke(
    query = "What is the best video game for driving, according to your database?" , 
    session_id = "video_game_2",
)

print("\nMessages from run 2:")
messages = run_2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are an Agentic RAG assistant that specializes in content about the video game industry.You can intelligently decide which tools to use to answer user questions.You have access to 3 tools: search_video_game_collection, web_search and evalutate_retrieval_toolWhen answering, first retrieve internal docs via search_video_game_collection, then evaluate results with evaluate_retrieval_tool.If not useful, take the recommended actions in order: (web_search, rewrite query or use foundational knowledge).Reason about about the response, change the query and call the tool again if needed in order to g

In [23]:
run_2 = agentic_rag.invoke(
    query = "List the upcoming video game trends in 2027 for Virtual Reality" , 
    session_id = "video_game_2",
)

print("\nMessages from run 2:")
messages = run_2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are an Agentic RAG assistant that specializes in content about the video game industry.You can intelligently decide which tools to use to answer user questions.You have access to 3 tools: search_video_game_collection, web_search and evalutate_retrieval_toolWhen answering, first retrieve internal docs via search_video_game_collection, then evaluate results with evaluate_retrieval_tool.If not useful, take the recommended actions in order: (web_search, rewrite query or use foundational knowledge).Reason about about the response, change the query and call the tool again if needed in order to get better results.Always explain your reasoning for tool selection and provide comprehensi

```
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?
```

In [24]:
video_game_test_queries = [
    "When was the game Pokemon Gold and Silver was released?", 
    "Which one was the first 3D platformer Mario game?", 
    "Was Mortal Kombat X realeased for Playstation 5?", # Mortal Kombat X is not in the Vector DB
    "What is the earliest and latest Gran Turismo games?", 
    "What are some fighting games released in the year 2001?", 
    "Do a comparison of the game Halo Infinite and the latest Gears of War game"
]

for query in video_game_test_queries: 

    run_3 = agentic_rag.invoke(
        query = query , 
        session_id = "video_game_3",
    )

print("\nMessages from run 3:")
messages = run_3.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor


## Analysis of Results 
Query: `What is the best video game for driving, according to your database?`
Analysis: The agent did a query of the RAG DB and found the data useful. However, the agent is able to reason that the data can be augmented by online searches. The agent presented doing the web search as an option to the user, along with the database results. 

Query: `List the upcoming video game trends in 2027 for Virtual Reality`
Analysis: The agent is able to reason that this information is not in the RAG DB and did a web search instead. 

Query: `When was the game Pokemon Gold and Silver was released`
Analysis: The agent queried the RAG DB and obtained the release year as 1999. 

Query: `Which one was the first 3D platformer Mario game?`     
Analysis: The agent queried the RAG DB and evaluated that the information was useful. Hence, it returned the answer as Super Mario 64. 

Query: `Was Mortal Kombat X released for PlayStation 5?`
Analysis: Mortal Kombat was not stored in the RAG DB. Hence, it was evaluated as not useful and the agent had to fall back to the web search. 

Query: `What is the earliest and latest Gran Turismo games?`
Analysis: The agent first queried the RAG DB to find the earliest and latest Gran Turismo game. However, the agent reasoned that the latest game may not be in the DB. Hence, it did a web search and was able to find the latest Gran Turismo game. 

Query: `What are some fighting games released in the year 2001?`
Analysis: The agent first queried the RAG DB but only found 1 fighting game. It falls back to using the web search to augment the response with other games such as King of Fighters. 

Query: `Do a comparison of the game Halo Infinite and the latest Gears of War game`
Analysis: The agent is able to find information about Halo Infinite from the RAG DB but not Gears of War. Hence, it did a web search for the Gears of War game and is able to present the full comparison to the user.



### (Optional) Advanced

In [30]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes